# 상담 데이터 만들기 

### 상담 데이터 예시 

```
상담사: "안녕하세요, 고객님. 하나은행 김철수입니다. 무엇을 도와드릴까요?"

고객: "안녕하세요, 김철수 상담사님. 제 이름은 박민수입니다. 최근에 적금 상품에 대해 상담을 받고 싶어서요. 제 생일은 1985년 7월 15일이고, 계좌번호는 110-234-567890입니다. 서울 용산구 후암동에 살고 있어요."

상담사: "네, 박민수 고객님. 적금 상품에 대해 관심이 있으시군요. 혹시 연락 가능한 번호는 010-9876-5432 맞으신가요?"

고객: "네, 맞습니다. 요즘 고금리 적금 상품이 있다고 들었는데, 어떤 상품을 추천하시겠어요?"

상담사: "확인해 보겠습니다. 현재 저희 은행에서는 연 2.5% 금리의 '희망 적금' 상품이 인기가 많습니다. 가입 기간은 1년이며, 월 최대 50만 원까지 적립하실 수 있습니다."

고객: "그렇군요. 그럼, 만약 가입하고 싶은 경우 필요한 서류나 절차가 있을까요?"

상담사: "네, 주민등록증이나 운전면허증 같은 신분증이 필요하고, 서명 절차를 위해 직접 지점을 방문해 주셔야 합니다. 이메일 주소를 남겨주시면 자세한 내용과 절차를 안내해 드리겠습니다."

고객: "좋아요. 이메일은 minspark85@gmail.com입니다."

상담사: "알겠습니다, 박민수 고객님. 이메일로 자세한 정보를 보내드리겠습니다. 추가로 궁금한 점이 있으면 언제든지 연락 주세요. 감사합니다."

고객: "네, 감사합니다. 좋은 하루 되세요."
```

### 비식별화 데이터 예시 

```
상담사: "안녕하세요, 고객님. 하나은행 [PERSON1]입니다. 무엇을 도와드릴까요?"

고객: "안녕하세요, [PERSON1] 상담사님. 제 이름은 [PERSON2]입니다. 최근에 적금 상품에 대해 상담을 받고 싶어서요. 제 생일은 [DATEOFBIRTH1]이고, 계좌번호는 [ACCOUNT1]입니다. [ADDRESS1]에 살고 있어요."

상담사: "네, [PERSON2] 고객님. 적금 상품에 대해 관심이 있으시군요. 혹시 연락 가능한 번호는 [CONTACT1] 맞으신가요?"

고객: "네, 맞습니다. 요즘 고금리 적금 상품이 있다고 들었는데, 어떤 상품을 추천하시겠어요?"

상담사: "확인해 보겠습니다. 현재 저희 은행에서는 연 2.5% 금리의 '희망 적금' 상품이 인기가 많습니다. 가입 기간은 1년이며, 월 최대 50만 원까지 적립하실 수 있습니다."

고객: "그렇군요. 그럼, 만약 가입하고 싶은 경우 필요한 서류나 절차가 있을까요?"

상담사: "네, 주민등록증이나 운전면허증 같은 신분증이 필요하고, 서명 절차를 위해 직접 지점을 방문해 주셔야 합니다. 이메일 주소를 남겨주시면 자세한 내용과 절차를 안내해 드리겠습니다."

고객: "좋아요. 이메일은 [EMAIL1]입니다."

상담사: "알겠습니다, [PERSON2] 고객님. 이메일로 자세한 정보를 보내드리겠습니다. 추가로 궁금한 점이 있으면 언제든지 연락 주세요. 감사합니다."

고객: "네, 감사합니다. 좋은 하루 되세요."
```

In [1]:
import re
import os

import json_repair
import pandas as pd
from openai import OpenAI
from pydantic import BaseModel
from dotenv import load_dotenv
from pqdm.processes import pqdm
from datasets import load_dataset

# 환경변수 로드
load_dotenv("./credit-env")
client = OpenAI(api_key=os.getenv("SELF_OPENAI_API_KEY"))

# 데이터셋 로드
ds = load_dataset("daekeun-ml/naver-news-summarization-ko")
ds

/home/admin/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [2]:
ds["train"][0]

{'date': '2022-07-03 17:14:37',
 'category': 'economy',
 'press': 'YTN ',
 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대',
 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 'link': 'https://n.news.naver.com/mne

In [3]:
# config들 설정
## ✅ 카테고리 리스트 설정
categories = ["은행 상담", "보험 상담", "쇼핑 고객 서비스"]


## ✅ 데이터 개수 설정
num_samples = 2  # 한 카테고리당 몇 개 생성할지 설정
N_JOBS = 2

## ✅ Reference 데이터 가져오기
reference_data_list = ds["train"]["title"][:num_samples]


In [4]:
# 데이터 생성
### ✅ 데이터 생성 프롬프트
data_system_prompt = f"""당신은 한국어로 다양한 카테고리의 현실적이고 자연스러운 데이터를 생성하는 어시스턴트입니다."""

data_user_prompt = f"""
{{category}}에 대해 현실적이고 자연스러운 데이터를 작성하세요.

# Output Format
상담사: "..."
             
고객: "..."
             
상담사: "..."
             
고객: "..."
             
...


각 데이터는 실제로 존재할 법한 대화 형태로 작성되며, 반드시 아래의 개인정보 중 최소 3가지 이상을 포함해야 합니다.

- 사람 이름
- 생년월일
- 연락처 (전화번호, 이메일, 카카오톡 ID 등)
- 주소
- 계좌번호
- 소셜미디어 ID (트위터, 텔레그램 등)

작성할 때 개인정보(이름, 연락처, 생년월일, 계좌번호, 주소, 이메일 등)를 구체적으로 포함해야 합니다. 
데이터의 전체 길이는 최소 300자 이상, 최대 1000자 이하로 작성합니다.
실제로 존재할 법한 내용과 문맥을 갖추어 자연스럽게 작성합니다.
마크다운 포맷으로 작성하지 마세요. 
아래 reference로부터 아이디어를 얻어 {{category}}에 대해 현실적이고 자연스러운 데이터를 작성하세요.

reference:::
{{reference_data}}
reference:::

자! 시작
"""

### 1️⃣ 데이터 생성 함수
def generate_data(args):
    category, reference_data = args
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": data_system_prompt},
            {"role": "user", "content": data_user_prompt.format(category=category, reference_data=reference_data)}
        ],
        temperature=0.9
    )
    return {
        "data": response.choices[0].message.content, 
        "category": category, 
        "prompt": data_user_prompt.format(
            category=category, 
            reference_data=reference_data)
            }

### ✅ 병렬 데이터 생성
print(f"{', '.join(categories)} 주어진 카테고리에 대해 데이터 생성 중...")
generated_data_list = pqdm(
    [(category, reference_data) for category in categories for reference_data in reference_data_list],
    generate_data,
    n_jobs=N_JOBS
)

### 데이터 생성 결과 예시
generated_data_list[:2]

은행 상담, 보험 상담, 쇼핑 고객 서비스 주어진 카테고리에 대해 데이터 생성 중...


QUEUEING TASKS | : 100%|██████████| 600/600 [00:00<00:00, 3188.87it/s]
PROCESSING TASKS | : 100%|██████████| 600/600 [04:58<00:00,  2.01it/s]
COLLECTING RESULTS | : 100%|██████████| 600/600 [00:00<00:00, 234035.38it/s]


[{'data': '상담사: "안녕하세요, 고객님. 하나은행 김철수입니다. 무엇을 도와드릴까요?"\n\n고객: "안녕하세요, 김철수 상담사님. 제 이름은 박민수입니다. 최근에 적금 상품에 대해 상담을 받고 싶어서요. 제 생일은 1985년 7월 15일이고, 계좌번호는 110-234-567890입니다. 서울 용산구 후암동에 살고 있어요."\n\n상담사: "네, 박민수 고객님. 적금 상품에 대해 관심이 있으시군요. 혹시 연락 가능한 번호는 010-9876-5432 맞으신가요?"\n\n고객: "네, 맞습니다. 요즘 고금리 적금 상품이 있다고 들었는데, 어떤 상품을 추천하시겠어요?"\n\n상담사: "확인해 보겠습니다. 현재 저희 은행에서는 연 2.5% 금리의 \'희망 적금\' 상품이 인기가 많습니다. 가입 기간은 1년이며, 월 최대 50만 원까지 적립하실 수 있습니다."\n\n고객: "그렇군요. 그럼, 만약 가입하고 싶은 경우 필요한 서류나 절차가 있을까요?"\n\n상담사: "네, 주민등록증이나 운전면허증 같은 신분증이 필요하고, 서명 절차를 위해 직접 지점을 방문해 주셔야 합니다. 이메일 주소를 남겨주시면 자세한 내용과 절차를 안내해 드리겠습니다."\n\n고객: "좋아요. 이메일은 minspark85@gmail.com입니다."\n\n상담사: "알겠습니다, 박민수 고객님. 이메일로 자세한 정보를 보내드리겠습니다. 추가로 궁금한 점이 있으면 언제든지 연락 주세요. 감사합니다."\n\n고객: "네, 감사합니다. 좋은 하루 되세요."',
  'category': '은행 상담',
  'prompt': '\n은행 상담에 대해 현실적이고 자연스러운 데이터를 작성하세요.\n\n# Output Format\n상담사: "..."\n             \n고객: "..."\n             \n상담사: "..."\n             \n고객: "..."\n             \n...\n\n\n각 데이터는 실제로 존재할 법한 대화 형태로 작성되며, 반드시 아래의 개인

혹시 실험을 따라하시다가 이런 에러가 났다면, OpenAI API 충전금액을 확인해주세요.  

concurrent.futures.process.BrokenProcessPool('A process in the process pool was terminated abruptly while the future was running or pending.')

In [5]:
# 글에 비식별화 정량평가에 사용할 []가 있는 데이터는 제거 
original_data_len = len(generated_data_list)
print(f"생성된 데이터 수 : {len(generated_data_list)}")
# 저희는 []를 활용하여 비식별화 되는 데이터를 찾을 예정이기에 데이터를 생성할 때 []가 포함된 데이터는 제거
generated_data_list = [article for article in generated_data_list if "[" not in article["data"]]
print(f"필터링된 데이터 수 : {len(generated_data_list)}")
print(f"필터링된 데이터수 : {original_data_len - len(generated_data_list)}")

생성된 데이터 수 : 600
필터링된 데이터 수 : 599
필터링된 데이터수 : 1


In [6]:
# OutputFormat 클래스는 OpenAI의 "structured data" 응답을 받을 때 사용할 데이터 형식 정의 
# OpenAI의 structured response_format을 사용할 때 응답을 json 형태로 받기 위해 사용하는 클래스
# API 호출 시 이 클래스에서 지정한 형태의 데이터를 반환하도록 GPT 모델에게 지시할 수 있음 
# 여기서는 평가 점수(score)와 평가 이유(reason)를 받음

class OutputFormat(BaseModel):
    score: int  # 데이터 품질 평가 점수 (1~5)
    reason: str  # 해당 점수를 매긴 이유 (한국어로 설명)

In [7]:
### 2️⃣ 데이터 평가 함수

data_valid_system_prompt = "당신은 프롬프트와 데이터 품질 평가하는 어시스턴트입니다."

data_valid_user_prompt = f"""주어진 프롬프트에 맞게 데이터가 생성되었는지 평가하세요.

# 프롬프트
{{prompt}}


# 데이터
{{data}}

# 데이터 평가 기준:
- 개인정보 포함: 이름, 생년월일, 연락처 등 최소 3가지 이상의 개인정보가 포함되었는가? 개인정보를 많이 가지고 있으면 있을수록 좋습니다.
- 개인정보 포맷: 개인정보는 현실에서 나올 법한 포맷을 갖추고 있는가?
- 현실적 대화: 생성된 대화는 현실에서 나올 법한 자연스러운 문맥을 갖추고 있는가?
- 문장의 길이: 데이터는 최소 300자 이상, 최대 1000자 이하의 길이를 갖추었는가?

# 평가 점수:
- 모든 조건을 완벽히 만족: 5점
- 1개의 평가 조건이 부족함: 4점
- 2개의 평가 조건이 부족함: 3점
- 3개의 평가 조건이 부족함: 2점
- 4개의 평가 조건이 부족함: 1점

OUTPUT은 평가 점수와 그렇게 평가한 이유를 한국어로 출력하세요."""


def validate_data(args):
    import json_repair
    data, category, prompt = args
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": data_valid_system_prompt},
            {"role": "user", "content": data_valid_user_prompt.format(prompt=prompt, data=data)}
        ],
        temperature=0.5,
        response_format=OutputFormat
    )
    result = json_repair.loads(response.choices[0].message.content)
    
    return {
        "generated_data": data,
        "category": category, 
        "generate_score": result["score"], 
        "generate_reason": result["reason"], 
        }

# ✅ 병렬 데이터 검증
print(f"{', '.join(categories)} 주어진 카테고리에 대해 데이터 검증 중...")
validated_data_list = pqdm(
    [(args["data"], args["category"], args["prompt"]) for args in generated_data_list],
    validate_data,
    n_jobs=N_JOBS
)

# ✅ score 5인 데이터만 선별
high_quality_data = [item for item in validated_data_list if item["generate_score"] == 5]

print("5점 이상의 데이터 개수:", len(high_quality_data))
print("5점 이상의 데이터 예시:")
high_quality_data[:2]

은행 상담, 보험 상담, 쇼핑 고객 서비스 주어진 카테고리에 대해 데이터 검증 중...


QUEUEING TASKS | : 100%|██████████| 599/599 [00:00<00:00, 7285.81it/s]
PROCESSING TASKS | : 100%|██████████| 599/599 [10:25<00:00,  1.04s/it]
COLLECTING RESULTS | : 100%|██████████| 599/599 [00:00<00:00, 233993.49it/s]

5점 이상의 데이터 개수: 574
5점 이상의 데이터 예시:


[{'generated_data': '상담사: "안녕하세요, 고객님. 하나은행 김철수입니다. 무엇을 도와드릴까요?"\n\n고객: "안녕하세요, 김철수 상담사님. 제 이름은 박민수입니다. 최근에 적금 상품에 대해 상담을 받고 싶어서요. 제 생일은 1985년 7월 15일이고, 계좌번호는 110-234-567890입니다. 서울 용산구 후암동에 살고 있어요."\n\n상담사: "네, 박민수 고객님. 적금 상품에 대해 관심이 있으시군요. 혹시 연락 가능한 번호는 010-9876-5432 맞으신가요?"\n\n고객: "네, 맞습니다. 요즘 고금리 적금 상품이 있다고 들었는데, 어떤 상품을 추천하시겠어요?"\n\n상담사: "확인해 보겠습니다. 현재 저희 은행에서는 연 2.5% 금리의 \'희망 적금\' 상품이 인기가 많습니다. 가입 기간은 1년이며, 월 최대 50만 원까지 적립하실 수 있습니다."\n\n고객: "그렇군요. 그럼, 만약 가입하고 싶은 경우 필요한 서류나 절차가 있을까요?"\n\n상담사: "네, 주민등록증이나 운전면허증 같은 신분증이 필요하고, 서명 절차를 위해 직접 지점을 방문해 주셔야 합니다. 이메일 주소를 남겨주시면 자세한 내용과 절차를 안내해 드리겠습니다."\n\n고객: "좋아요. 이메일은 minspark85@gmail.com입니다."\n\n상담사: "알겠습니다, 박민수 고객님. 이메일로 자세한 정보를 보내드리겠습니다. 추가로 궁금한 점이 있으면 언제든지 연락 주세요. 감사합니다."\n\n고객: "네, 감사합니다. 좋은 하루 되세요."',
  'category': '은행 상담',
  'generate_score': 5,
  'generate_reason': '제공된 데이터는 다음 평가 기준을 모두 만족합니다: \n\n1. **개인정보 포함**: 이름(박민수), 생년월일(1985년 7월 15일), 연락처(010-9876-5432), 계좌번호(110-234-567890), 주소(서울 용산구 후암동), 이메일(minspark85@gmail.com) 등 6가지 개인

In [8]:
anonymized_system_prompt = f"""당신은 한국어로 데이터 내에 포함된 모든 개인정보를 placeholder로 비식별화하는 작업을 수행하는 어시스턴트입니다."""

anonymized_user_prompt = f"""입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 처리합니다. 동일 인물의 개인정보는 같은 번호를 사용하여 일관성을 유지해야 합니다.
입력 데이터를 바탕으로, 아래 placeholder를 사용하여 모든 개인정보를 비식별화하세요.

intput:::
{{text}}
intput:::

| 개인정보 종류 | placeholder 예시       |
|---------------|------------------------|
| 이름          | `[PERSON1]`, `[PERSON2]` 등 |
| 생년월일      | `[DATEOFBIRTH1]`, `[DATEOFBIRTH2]` 등 |
| 연락처        | `[CONTACT1]`, `[CONTACT2]` 등 |
| 주소          | `[ADDRESS1]`, `[ADDRESS2]` 등 |
| 계좌번호      | `[ACCOUNT1]`, `[ACCOUNT2]` 등 |
| 이메일        | `[EMAIL1]`, `[EMAIL2]` 등 |
| 장소 및 지역명 | `[LOCATION1]`, `[LOCATION2]` 등 |
| 카카오톡 ID   | `[KAKAO_ID1]`, `[KAKAO_ID2]` 등 |
| 트위터 ID     | `[TWITTER_ID1]`, `[TWITTER_ID2]` 등 |
| 텔레그램 ID   | `[TELEGRAM_ID1]`, `[TELEGRAM_ID2]` 등 |


자! 시작
"""

### 3️⃣ 비식별화 함수
def anonymize_data(args):
    generated_data, category, generate_score, generate_reason = args
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": anonymized_system_prompt},
            {"role": "user", "content": anonymized_user_prompt.format(text=generated_data)}
        ],
        temperature=0.1
    )
    return {
        "generated_data": generated_data, 
        "category": category, 
        "generate_score": generate_score, 
        "generate_reason": generate_reason,
        "anonymized_data": response.choices[0].message.content, 
        "anonymized_prompt": anonymized_user_prompt.format(text=generated_data)
        }

# ✅ 병렬 비식별화 처리
anonymized_data_list = pqdm(
    [(
        item["generated_data"], 
        item["category"], 
        item["generate_score"], 
        item["generate_reason"]
        ) for item in high_quality_data],
    anonymize_data,
    n_jobs=N_JOBS
)


QUEUEING TASKS | : 100%|██████████| 574/574 [00:00<00:00, 7623.18it/s]
PROCESSING TASKS | : 100%|██████████| 574/574 [18:05<00:00,  1.89s/it]
COLLECTING RESULTS | : 100%|██████████| 574/574 [00:00<00:00, 157221.35it/s]


In [9]:
anonymized_data_list[:1]

[{'generated_data': '상담사: "안녕하세요, 고객님. 하나은행 김철수입니다. 무엇을 도와드릴까요?"\n\n고객: "안녕하세요, 김철수 상담사님. 제 이름은 박민수입니다. 최근에 적금 상품에 대해 상담을 받고 싶어서요. 제 생일은 1985년 7월 15일이고, 계좌번호는 110-234-567890입니다. 서울 용산구 후암동에 살고 있어요."\n\n상담사: "네, 박민수 고객님. 적금 상품에 대해 관심이 있으시군요. 혹시 연락 가능한 번호는 010-9876-5432 맞으신가요?"\n\n고객: "네, 맞습니다. 요즘 고금리 적금 상품이 있다고 들었는데, 어떤 상품을 추천하시겠어요?"\n\n상담사: "확인해 보겠습니다. 현재 저희 은행에서는 연 2.5% 금리의 \'희망 적금\' 상품이 인기가 많습니다. 가입 기간은 1년이며, 월 최대 50만 원까지 적립하실 수 있습니다."\n\n고객: "그렇군요. 그럼, 만약 가입하고 싶은 경우 필요한 서류나 절차가 있을까요?"\n\n상담사: "네, 주민등록증이나 운전면허증 같은 신분증이 필요하고, 서명 절차를 위해 직접 지점을 방문해 주셔야 합니다. 이메일 주소를 남겨주시면 자세한 내용과 절차를 안내해 드리겠습니다."\n\n고객: "좋아요. 이메일은 minspark85@gmail.com입니다."\n\n상담사: "알겠습니다, 박민수 고객님. 이메일로 자세한 정보를 보내드리겠습니다. 추가로 궁금한 점이 있으면 언제든지 연락 주세요. 감사합니다."\n\n고객: "네, 감사합니다. 좋은 하루 되세요."',
  'category': '은행 상담',
  'generate_score': 5,
  'generate_reason': '제공된 데이터는 다음 평가 기준을 모두 만족합니다: \n\n1. **개인정보 포함**: 이름(박민수), 생년월일(1985년 7월 15일), 연락처(010-9876-5432), 계좌번호(110-234-567890), 주소(서울 용산구 후암동), 이메일(minspark85@gmail.com) 등 6가지 개인

In [10]:
anonymized_valid_system_prompt = "당신은 비식별화 데이터 품질 평가하는 어시스턴트입니다."

anonymized_valid_user_prompt = f"""당신에게 비식별화 프롬프트와 비식별화 데이터의 품질을 평가하세요. 

# 비식별화 프롬프트
{{prompt}}


# 비식별화 데이터
{{data}}

origin_data
# 검증 기준:  
1. 모든 개인정보가 비식별화되었는가?  
   - 이름, 생년월일, 연락처, 주소, 계좌번호 등 모든 개인정보가 placeholder로 대체되었는가?  
2. 동일한 개인정보는 일관된 placeholder를 사용하는가?  
   - 같은 인물, 같은 이메일, 같은 연락처 등이 일관된 ID로 처리되었는가?  
3. 비식별화 후 문장이 자연스러운가?  
   - 문맥이 손상되지 않고, 대화 흐름이 유지되는가?  

# 평가 점수:  
- 5점: 모든 기준을 완벽하게 충족 (개인정보가 누락 없이 비식별화되었고, 일관성이 유지되며 문장이 자연스러움)  
- 4점: 거의 완벽하지만 일부 개인정보가 누락되었거나 일관성이 살짝 부족함  
- 3점: 여러 개인정보가 비식별화되지 않았거나, 동일 개인정보의 placeholder가 일관되지 않음  
- 2점: 대부분의 개인정보가 비식별화되지 않았으며, placeholder 사용이 올바르지 않음  
- 1점: 개인정보가 거의 그대로 남아 있거나, 잘못된 방식으로 변환됨

OUTPUT은 평가 점수와 그렇게 평가한 이유를 한국어로 출력하세요."""

### 4️⃣ 비식별화 검증 함수
def validate_anonymized_data(args):
    ( 
        generated_data, 
        category, 
        generate_score, 
        generate_reason, 
        anonymized_data,
        anonymized_prompt  ) = args
    ####################
    ### beta 모델 호출 ###
    ##################
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": anonymized_valid_system_prompt},
            {"role": "user", "content": anonymized_valid_user_prompt.format(
                prompt=anonymized_prompt, 
                data=anonymized_data
                )
            }
        ],
        temperature=0.4,
        response_format=OutputFormat
    )
    result = response.choices[0].message.content
    ################
    ### json 파싱 ###
    ################
    result = json_repair.loads(result)
    return {
                "origin_data": generated_data, 
                "category": category, 
                "generate_score": generate_score,
                "generate_reason": generate_reason,
                "anonymized_data": anonymized_data,
                "anonymized_prompt": anonymized_prompt,
                "validate_score": result["score"], 
                "validate_reason": result["reason"]
            }

# ✅ 병렬 비식별화 검증
validated_anonymized_data_list = pqdm(
    [(
        item["generated_data"], 
        item["category"], 
        item["generate_score"],
        item["generate_reason"], 
        item["anonymized_data"], 
        item["anonymized_prompt"] ) for item in anonymized_data_list],
    validate_anonymized_data,
    n_jobs=N_JOBS
)

QUEUEING TASKS | : 100%|██████████| 574/574 [00:00<00:00, 2100.81it/s]
PROCESSING TASKS | : 100%|██████████| 574/574 [01:46<00:00,  5.37it/s]
COLLECTING RESULTS | : 100%|██████████| 574/574 [00:00<00:00, 217207.73it/s]


In [11]:
validated_anonymized_data_list[1]

{'origin_data': '상담사: "안녕하세요, 우리 은행입니다. 무엇을 도와드릴까요?"\n\n고객: "안녕하세요, 제 이름은 김민수이고 계좌번호는 110-234-567890입니다. 이번 달 카드 명세서가 잘못된 것 같아서요."\n\n상담사: "확인해 드리겠습니다, 김민수님. 생년월일과 연락처를 알려주실 수 있을까요?"\n\n고객: "네, 1990년 5월 10일 생이고, 연락처는 010-1234-5678입니다."\n\n상담사: "감사합니다. 확인해 보니 지난 달에 해외 결제가 몇 건 있었네요. 혹시 해당 결제를 하신 기억이 있으신가요?"\n\n고객: "해외 결제는 하지 않았는데요. 주소는 대전시 서구 둔산동 123-45번지입니다."\n\n상담사: "알겠습니다. 제가 해외 결제 건에 대해 분쟁 접수를 해 드리도록 하겠습니다. 이메일 주소를 알려주시면 처리 결과를 보내드리겠습니다."\n\n고객: "이메일은 minsukim@gmail.com이에요."\n\n상담사: "네, 이메일로 최대한 빠르게 처리 결과를 보내드리겠습니다. 계좌 안전을 위해 인터넷 뱅킹 비밀번호를 변경해 주시고, 카드도 새로 발급 받으시는 것이 좋을 것 같습니다."\n\n고객: "알겠습니다. 빠른 확인 부탁드릴게요. 감사합니다."\n\n상담사: "네, 처리 후 다시 연락드리겠습니다. 좋은 하루 되세요!"',
 'category': '은행 상담',
 'generate_score': 5,
 'generate_reason': '제공된 데이터는 모든 평가 기준을 완벽히 만족합니다. \n\n1. 개인정보 포함: 이름, 생년월일, 연락처, 계좌번호, 이메일, 주소 등 총 6가지의 개인정보가 포함되어 있습니다.\n2. 개인정보 포맷: 제공된 개인정보는 현실에서 사용되는 형식을 잘 따르고 있습니다. 예를 들어, 전화번호는 한국의 일반적인 형식을 따르고 있습니다.\n3. 현실적 대화: 상담사와 고객 간의 대화는 은행 상담에서 실제로 있을 법한 상황과 내용으로 자연스럽게 구성되어 있습니다.\n4. 문장의 길이:

In [12]:
print("검증데이터 개수:", len(validated_anonymized_data_list))
# ✅ 최종 데이터 선별 (비식별화 평가 5점만)
final_data = [item for item in validated_anonymized_data_list if item["validate_score"] == 5]
print("최종데이터 개수:", len(final_data))

검증데이터 개수: 574
최종데이터 개수: 450


In [13]:
import pandas as pd 
df = pd.DataFrame(final_data)
print("최종 데이터 개수:", df.shape)
df = df[(df["generate_score"] == 5) & (df["validate_score"] == 5)]
print("모두 5점 이상 데이터 개수:", df.shape)
df.head(2)

최종 데이터 개수: (450, 8)
모두 5점 이상 데이터 개수: (450, 8)


,origin_data,category,generate_score,generate_reason,anonymized_data,anonymized_prompt,validate_score,validate_reason
0,"상담사: ""안녕하세요, 고객님. 하나은행 김철수입니다. 무엇을 도와드릴까요?""\n\...",은행 상담,5,제공된 데이터는 다음 평가 기준을 모두 만족합니다: \n\n1. **개인정보 포함*...,"상담사: ""안녕하세요, 고객님. 하나은행 [PERSON1]입니다. 무엇을 도와드릴까...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,제공된 비식별화 데이터는 모든 개인정보가 적절한 placeholder로 대체되었으며...
1,"상담사: ""안녕하세요, 우리 은행입니다. 무엇을 도와드릴까요?""\n\n고객: ""안녕...",은행 상담,5,제공된 데이터는 모든 평가 기준을 완벽히 만족합니다. \n\n1. 개인정보 포함: ...,"상담사: ""안녕하세요, 우리 은행입니다. 무엇을 도와드릴까요?""\n\n고객: ""안녕...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,"모든 개인정보가 적절히 비식별화되었으며, 동일한 개인정보는 일관된 placehold..."


In [14]:

def extract_placeholder_mapping(original_text, transformed_text, allowed_types):
    allowed_pattern = re.compile(r'\[(' + '|'.join(allowed_types) + r')\d*\]')
    generic_pattern = re.compile(r'(\[[^]]+\])')

    mapping = {}

    orig_lines = original_text.splitlines()
    trans_lines = transformed_text.splitlines()
    n_lines = min(len(orig_lines), len(trans_lines))

    for idx in range(n_lines):
        orig_line = orig_lines[idx]
        trans_line = trans_lines[idx]

        parts = re.split(generic_pattern, trans_line)
        orig_pos = 0

        for i, part in enumerate(parts):
            if allowed_pattern.match(part):
                # placeholder 발견
                # 다음 literal을 찾음
                next_literal = parts[i + 1] if i + 1 < len(parts) else ''
                
                # 다음 literal이 존재하면, 그 literal까지의 텍스트를 추출
                if next_literal:
                    next_idx = orig_line.find(next_literal, orig_pos)
                    if next_idx != -1:
                        replaced_text = orig_line[orig_pos:next_idx]
                        orig_pos = next_idx
                    else:
                        # 다음 literal을 못 찾으면 끝까지
                        replaced_text = orig_line[orig_pos:]
                        orig_pos = len(orig_line)
                else:
                    # 다음 literal이 없으면 남은 텍스트 전체
                    replaced_text = orig_line[orig_pos:]
                    orig_pos = len(orig_line)

                replaced_text = replaced_text.strip()
                if replaced_text:
                    mapping[replaced_text] = part

            else:
                # literal인 경우, 원본에서 위치 업데이트
                found_idx = orig_line.find(part, orig_pos)
                if found_idx != -1:
                    orig_pos = found_idx + len(part)

    return mapping


extract_placeholder_mapping 함수의 결과 
- 입력 : (상담 데이터, 비식별화 데이터)
- 출력 
    * 김수진 -> [PERSON1]
    * 110-2034-5678 -> [ACCOUNT1]
    * 1985년 8월 17일 -> [DATEOFBIRTH1]
    * 010-2345-6789 -> [CONTACT1]
    * suji.kim85@gmail.com -> [EMAIL1]
    * 서울 강남구 도곡동 123-45 -> [ADDRESS1]
    * 1588-1234 -> [CONTACT2]

In [15]:
# mapping 예시 추출
mapping = extract_placeholder_mapping(
    df["origin_data"].iloc[0], 
    df["anonymized_data"].iloc[0],
    allowed_types=(
        "PERSON", "CONTACT", "ADDRESS", 
        "ACCOUNT", "DATEOFBIRTH", "EMAIL", 
        "LOCATION", "KAKO_ID", "TIWTTER_ID", "TELEGRAM_ID"
        )
)

for orig_val, ph in mapping.items():
    print(f"{orig_val} -> {ph}")


김철수 -> [PERSON1]
박민수 -> [PERSON2]
1985년 7월 15일 -> [DATEOFBIRTH1]
110-234-567890 -> [ACCOUNT1]
서울 용산구 후암동 -> [ADDRESS1]
010-9876-5432 -> [CONTACT1]
minspark85@gmail.com -> [EMAIL1]


In [16]:
df["mapping"] = df.apply(lambda x: extract_placeholder_mapping(
    x["origin_data"], 
    x["anonymized_data"], 
    allowed_types=(
        "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", 
        "EMAIL", "LOCATION", "KAKO_ID", "TIWTTER_ID", "TELEGRAM_ID")), 
    axis=1)

df.head(2)

,origin_data,category,generate_score,generate_reason,anonymized_data,anonymized_prompt,validate_score,validate_reason,mapping
0,"상담사: ""안녕하세요, 고객님. 하나은행 김철수입니다. 무엇을 도와드릴까요?""\n\...",은행 상담,5,제공된 데이터는 다음 평가 기준을 모두 만족합니다: \n\n1. **개인정보 포함*...,"상담사: ""안녕하세요, 고객님. 하나은행 [PERSON1]입니다. 무엇을 도와드릴까...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,제공된 비식별화 데이터는 모든 개인정보가 적절한 placeholder로 대체되었으며...,"{'김철수': '[PERSON1]', '박민수': '[PERSON2]', '1985..."
1,"상담사: ""안녕하세요, 우리 은행입니다. 무엇을 도와드릴까요?""\n\n고객: ""안녕...",은행 상담,5,제공된 데이터는 모든 평가 기준을 완벽히 만족합니다. \n\n1. 개인정보 포함: ...,"상담사: ""안녕하세요, 우리 은행입니다. 무엇을 도와드릴까요?""\n\n고객: ""안녕...",입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 ...,5,"모든 개인정보가 적절히 비식별화되었으며, 동일한 개인정보는 일관된 placehold...","{'김민수': '[PERSON1]', '110-234-567890': '[ACCOU..."


In [35]:
extract_placeholder_mapping(
    df["origin_data"].iloc[300], 
    df["anonymized_data"].iloc[300], 
    allowed_types=(
        "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", 
        "EMAIL", "LOCATION", "KAKO_ID", "TIWTTER_ID", "TELEGRAM_ID"))

{'홍길동': '[PERSON1]',
 '김혜진': '[PERSON2]',
 '1985년 3월 15일': '[DATEOFBIRTH1]',
 '010-1234-5678': '[CONTACT1]',
 '서울시 강남구 테헤란로 123, 수도타워 101호': '[ADDRESS1]',
 '2015년 7월 10일': '[DATEOFBIRTH2]',
 'gildong.hong@email.com': '[EMAIL1]'}

그러나, 모든 데이터가 이렇게 만들어졌는지는 알 수 없습니다.  
다음 데이터를 살펴보겠습니다.  

In [25]:
print(df["origin_data"].iloc[299])


상담사: "안녕하세요, 이민수 고객님. 해피 보험입니다. 보험 상담 요청 주셔서 연락드렸습니다. 오늘 상담 가능한 시간 괜찮으신가요?"

고객: "네, 괜찮습니다. 제가 보험에 대해 궁금한 점이 있어서요."

상담사: "네, 그럼 본격적으로 상담을 시작해 보겠습니다. 확인을 위해 고객님의 생년월일과 연락처 부탁드립니다."

고객: "네, 제 생년월일은 1987년 5월 15일이고, 연락처는 010-9876-5432입니다."

상담사: "확인 감사합니다. 고객님께서 주로 어떤 보험에 관심이 있으신가요? 건강보험, 자동차보험, 아니면 다른 종류가 있으신가요?"

고객: "건강보험에 대해서 좀 더 알고 싶습니다. 특히 입원비나 수술비 보장 부분이요."

상담사: "네, 이해했습니다. 입원비와 수술비 보장은 고객님께서 선택하시는 보험 상품에 따라 다릅니다. 예를 들어, 저희의 프리미엄 건강보험 상품은 최대 5000만 원까지 수술비와 1일당 10만 원의 입원비를 보장합니다."

고객: "그렇군요. 그럼 제가 기존에 가입해 있는 보험과 비교해 보고 싶습니다. 그런데 보험금 수령 시, 어떤 서류가 필요한가요?"

상담사: "기본적으로 진단서, 입퇴원 확인서, 의료비 영수증 등이 필요합니다. 추가적으로 요구되는 서류는 보험금 청구 시 안내드리고 있습니다."

고객: "알겠습니다. 그럼 좀 더 고민해 보고 다시 연락드리겠습니다."

상담사: "네, 언제든지 편하신 시간에 연락 주세요. 상담 도와드리겠습니다. 혹시 더 궁금하신 점 있으시면 제 이메일(jinsung.kim@happyins.com)로 문의 주셔도 됩니다."

고객: "감사합니다. 이메일로 문의 드리겠습니다."

상담사: "네, 오늘 상담 감사합니다. 좋은 하루 보내세요."


In [26]:
print(df["anonymized_data"].iloc[299])

상담사: "안녕하세요, [PERSON1] 고객님. 해피 보험입니다. 보험 상담 요청 주셔서 연락드렸습니다. 오늘 상담 가능한 시간 괜찮으신가요?"

고객: "네, 괜찮습니다. 제가 보험에 대해 궁금한 점이 있어서요."

상담사: "네, 그럼 본격적으로 상담을 시작해 보겠습니다. 확인을 위해 고객님의 생년월일과 연락처 부탁드립니다."

고객: "네, 제 생년월일은 [DATEOFBIRTH1]이고, 연락처는 [CONTACT1]입니다."

상담사: "확인 감사합니다. 고객님께서 주로 어떤 보험에 관심이 있으신가요? 건강보험, 자동차보험, 아니면 다른 종류가 있으신가요?"

고객: "건강보험에 대해서 좀 더 알고 싶습니다. 특히 입원비나 수술비 보장 부분이요."

상담사: "네, 이해했습니다. 입원비와 수술비 보장은 고객님께서 선택하시는 보험 상품에 따라 다릅니다. 예를 들어, 저희의 프리미엄 건강보험 상품은 최대 5000만 원까지 수술비와 1일당 10만 원의 입원비를 보장합니다."

고객: "그렇군요. 그럼 제가 기존에 가입해 있는 보험과 비교해 보고 싶습니다. 그런데 보험금 수령 시, 어떤 서류가 필요한가요?"

상담사: "기본적으로 진단서, 입퇴원 확인서, 의료비 영수증 등이 필요합니다. 추가적으로 요구되는 서류는 보험금 청구 시 안내드리고 있습니다."

고객: "알겠습니다. 그럼 좀 더 고민해 보고 다시 연락드리겠습니다."

상담사: "네, 언제든지 편하신 시간에 연락 주세요. 상담 도와드리겠습니다. 혹시 더 궁금하신 점 있으시면 제 이메일([EMAIL1])로 문의 주셔도 됩니다."

고객: "감사합니다. 이메일로 문의 드리겠습니다."

상담사: "네, 오늘 상담 감사합니다. 좋은 하루 보내세요."


In [29]:
extract_placeholder_mapping(
    df["origin_data"].iloc[299], 
    df["anonymized_data"].iloc[299], 
    allowed_types=(
        "PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", 
        "EMAIL", "LOCATION", "KAKO_ID", "TIWTTER_ID", "TELEGRAM_ID"))

{}

결과를 살펴보면, 어떤 정보도 placeholder와 매핑되어 있지 않습니다.  
왜 이런 결과가 나오는 걸까요? 바로 이 데이터의 line 수가 다릅니다. 

In [31]:
orig_lines = df["origin_data"].iloc[299].splitlines()
trans_lines = df["anonymized_data"].iloc[299].splitlines()
len(orig_lines), len(trans_lines)

(26, 25)

이처럼 비식별화의 깐깐한 평가기준을 통과했다고 하더라도, gpt-4o의 생성의 오류가 있을 수 있습니다.  
이에, 데이터를 생성할 때는 꼼꼼하게 데이터를 살펴보는 것이 중요합니다.

In [34]:
final_df = df[df["mapping"] != {}]
final_df.shape

(449, 9)

앞서 600개의 데이터를 생성하였는데, 최종적으로 엄격한 평가 기준과 449개의 데이터가 생성됩니다.

In [36]:
import time
for category in categories:
    sub_df = final_df[final_df["category"] == category]
    with open(
        f"./data/{category}_final_{time.strftime('%Y%m%d_%H%M%S')}.csv", 
        "w", 
        encoding="utf-8") as f:
        f.write(sub_df.to_csv(index=False))

print("✅ 모든 카테고리 데이터 처리 완료 🚀")

✅ 모든 카테고리 데이터 처리 완료 🚀
